# Pip Install

In [ ]:
pip install -r requirements.txt

# Import Library

In [ ]:
import sqlite3

import pandas as pd
import numpy as np

# 1. Preprocess and Clean

## Read Parquet File

In [ ]:
sample_df = pd.read_parquet("data/sample.parquet")

# remove rows any null value
sample_df = sample_df.dropna()

## Check time columns

The time string is processed here. It seems that it is some variant of ISO8601, but for some cases the second decimals are not zeropadded with a fix length. A regex match is ran to ensure there is no outlier format. 

An potential improvement is to stored time in epoch_ms from the start. There is no point for time to be human readable at the point, so it is much better to store time simply as epoch in milliseconds. This can improve data integrity check, remove string format ambiguity, and sorting and storing efficency. 

In [ ]:
# keep only acceptable time string format
time_regex = r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(?:\.\d+)?Z"
sample_df = sample_df.loc[sample_df["time"].str.match(time_regex)]

# convert time column to epoch_ms
sample_df["time"] = pd.to_datetime(sample_df['time'], format = "mixed")
sample_df['time'] = (sample_df['time'].astype(np.int64) // 10**6).astype(np.int64)

display(sample_df)

## keep only acceptable value for the rest of the columns

the accepted value is kept in a dict. For each columns names, it has to have a "dtype" attribute, and can have an optional list of "accepted_values"

Ex: 

```json
{
    "col_name_1": {
        "dtype": "dtype", 
        "accepted_values": [
            "accpeted_value_1",
            "accpeted_value_2",
            ...
            "accepted_value_x"
        ]
    }...
}
```

For any additional columns in the future, simply adding them to the dictionary. Additional, the dictionary can be store as a json config file for easy reading and editing. 

robot_id is converted to a string becuase later it needs to be transposed to a column names. For other use cases, it is best to keep it an int. 

I am not too sure why run_uuid is stored in a float64 instead of int64. Kept it same as in sample.parquet

An potential improvement is for any columns with a set of values, such as field and sensor_type, have an additional columns that stored them in an integer as instead. Ex. add an column called "sensor_type_int", and store "encoder" as 0 and "load_cell" as 1. Having a int column can improve filtering and sort immersive comparing to a string columns

In [ ]:
col_configs = {
    "value": {
        "dtype": "float64"
    },
    "field": {
        "dtype": "string", 
        "accepted_values": [
            "x",
            "y",
            "z",
            "fx",
            "fy",
            "fz",
        ],
    },
    "robot_id": {
        "dtype": "string", 
    }, 
    "run_uuid": {
        "dtype": "float64", 
    },
    "sensor_type": {
        "dtype": "string", 
        "accepted_values": [
            "encoder", 
            "load_cell",
        ],
    }
}

# go through the col and only keep acceptable list
for col_name, config in col_configs.items(): 
    # set col dtype
    sample_df[col_name] = sample_df[col_name].astype(config["dtype"])

    # if the columns has a list of acceptable value, enforce it
    if "accepted_values" in col_configs[col_name]: 
        sample_df = sample_df.loc[sample_df[col_name].isin(config["accepted_values"])]

display(sample_df)

## Index and sort columns to improve filtering and searching

The specific columns to index can be adjusted depending on user case. For this specific workload, i chose to index all columns except value to improve searching time during pviot. 

In [ ]:
sample_df = sample_df\
    .set_index(["run_uuid", "robot_id", "sensor_type", "field", "time"])\
    .sort_index()

display(sample_df)